## Panorama Stitching using SuperGlue
SuperGlue network is a Graph Neural Network combined with an Optimal Matching layer that is trained to perform matching on two sets of sparse image features.

In [30]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
from matplotlib import cm
import torch
from models.matching import Matching
from models.utils import (compute_pose_error, compute_epipolar_error,
                          estimate_pose, make_matching_plot,
                          error_colormap, AverageTimer, pose_auc, read_image,
                          rotate_intrinsics, rotate_pose_inplane,
                          scale_intrinsics)

torch.set_grad_enabled(False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
# generating the necessary txt file to input for the super glue algorithm
input_folder = '/home/aboggaram/Desktop/Orchard_Robotics_challenge/EvaluationSetRGB'
image_names = sorted(os.listdir(input_folder))
image_paths = [os.path.join(input_folder, name) for name in image_names]
nms_radius = 5
keypoint_threshold = 0.05
max_keypoints = 2048
superglue = 'outdoor'
sinkhorn_iterations = 20
match_threshold = 0.2

config = {
        'superpoint': {
            'nms_radius': nms_radius,
            'keypoint_threshold': keypoint_threshold,
            'max_keypoints': max_keypoints
        },
        'superglue': {
            'weights': superglue,
            'sinkhorn_iterations': sinkhorn_iterations,
            'match_threshold': match_threshold,
        }
    }
matching = Matching(config).eval().to(device)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


In [41]:
def frame2tensor(frame, device):
    return torch.from_numpy(frame/255.).float()[None, None].to(device)


def preprocess_image(path, device):
    image = cv.imread(str(path), cv.IMREAD_COLOR)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    if image is None:
        return None, None, None
    w, h = image.shape[1], image.shape[0]
    gray = gray.astype('float32')
    inp = frame2tensor(gray, device)
    return image, inp


def get_matches(inp0, inp1, matching=matching):
	# Perform the matching.
	pred = matching({'image0': inp0, 'image1': inp1})
	pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
	kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
	matches, conf = pred['matches0'], pred['matching_scores0']
	out_matches = {'keypoints0': kpts0, 'keypoints1': kpts1,
					'matches': matches, 'match_confidence': conf}
	return out_matches


def stitch_images(left_image, right_image, matches):
    # Get the keypoints from the matches
    kpts0 = matches['keypoints0']
    kpts1 = matches['keypoints1']

    mats = matches['matches']
    conf = matches['match_confidence']
    # Keep the matching keypoints.
    valid = mats > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[mats[valid]]
    mconf = conf[valid]

    k = 50
    # visualize the top k matches
    topk_matches = np.argsort(mconf)[::-1][:k]
    topk_mkpts0 = mkpts0[topk_matches]
    topk_mkpts1 = mkpts1[topk_matches]

    color = cm.jet(mconf[topk_matches])
    text = [
                'SuperGlue',
                'Keypoints: {}:{}'.format(len(kpts0), len(kpts1)),
                'Top {} Matches: {}'.format(k, len(topk_mkpts0)),
            ]

    make_matching_plot(left_image, right_image,
                        kpts0, kpts1,
                        topk_mkpts0, topk_mkpts1,
                        path='matches_test.jpg',
                        text=text,
                        color=color,
                        show_keypoints=False)

    # Find the Homography from left to right image
    H, status = cv.findHomography(mkpts0, mkpts1, cv.RANSAC, 5.0)
    H_inv = np.linalg.inv(H)

    # Warp the right image to align with the left image
    h1, w1 = left_image.shape[:2]
    h2, w2 = right_image.shape[:2]
    result_width = w1 + w2  # Width of both images combined
    result_height = max(h1, h2)  # Height of the taller image

    warped_image = cv.warpPerspective(right_image, H_inv, (result_width, result_height))

    cv.imwrite('warped_image.jpg', warped_image)


    # Blend the images using gradient-based blending
    blended_image = blend_images(left_image, warped_image)
    cv.imwrite('blended_image.jpg', blended_image)
    return blended_image


def blend_images(image1, image2):
    # Calculate the gradient masks for both images
    mask1 = np.zeros_like(image1)
    mask1[:, :image1.shape[1] // 2, :] = 1
    mask2 = 1 - mask1

    # Calculate gradients
    gradient1 = cv.Sobel(image1, cv.CV_64F, 1, 0, ksize=5)
    gradient2 = cv.Sobel(image2, cv.CV_64F, 1, 0, ksize=5)

    # Calculate the weighted sum of the gradients
    blended_gradient = mask1 * gradient1 + mask2 * gradient2

    # Calculate the blended image
    blended_image = image1.copy()
    blended_image[:, image1.shape[1] // 2:, :] = image2[:, image1.shape[1] // 2:, :]
    blended_image[:, :image1.shape[1] // 2, :] = cv.convertScaleAbs(blended_gradient)

    return blended_image


def test_image_stitching():
    # Read and preprocess the images
    left_image, left_input = preprocess_image(image_paths[0], device)
    right_image, right_input = preprocess_image(image_paths[1], device)
    # Get the matches
    matches = get_matches(left_input, right_input)
    # Stitch the images
    stitched_image = stitch_images(left_image, right_image, matches)
    # # Display the stitched image
    # plt.figure(figsize=(20, 20))
    # stitched_image = cv.cvtColor(stitched_image, cv.COLOR_BGR2RGB)
    # plt.imshow(stitched_image)


In [42]:
test_image_stitching()

/home/aboggaram/miniconda3/envs/yolov8/lib/python3.9/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


For each keypoint in `keypoints0`, the `matches` array indicates the index of the matching keypoint in `keypoints1`, or `-1` if the keypoint is unmatched.

In [ ]:
def plotMatches(imageSet):
    plt.figure(figsize=(10,10))
    matched_points = cv.imread('or_panorama/output/{:03}_rgb_{:03}_rgb_matches.png'.\
                     format(imageSet, imageSet+1),cv.IMREAD_ANYCOLOR)
    plt.imshow(matched_points, cmap='gray', vmin = 0, vmax = 255)
    plt.show()

In [ ]:
# opencv stitcher
import cv as cv
import os
stitcher = cv.Stitcher.create()
input_folder = '/home/aboggaram/Desktop/Orchard_Robotics_challenge/EvaluationSetRGB'
seq_image_paths = [input_folder + '/' + image for image in os.listdir(input_folder)]
seq_images = [cv.imread(image) for image in seq_image_paths]
status, stitched = stitcher.stitch(seq_images)
if status == cv.STITCHER_OK:
	plt.figure(figsize=(10,10))
	stitched = cv.cvtColor(stitched, cv.COLOR_BGR2RGB)
	plt.imshow(stitched)
	plt.show()


In [ ]:
import cv
import numpy as np

# Load the sequence of images
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg", ...]  # Replace with your image file paths
images = [cv.imread(path) for path in image_paths]

# Initialize an empty canvas for stitching
stitched_image = images[0]

# Iterate over the remaining images and stitch them together
for i in range(1, len(images)):
    # Detect and compute keypoints and descriptors
    orb = cv.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(stitched_image, None)
    keypoints2, descriptors2 = orb.detectAndCompute(images[i], None)

    # Perform feature matching
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Select the top N matches (you can adjust this number)
    N = 50
    matches = matches[:N]

    # Extract matching keypoints from both images
    src_pts = np.float32([keypoints1[match.queryIdx].pt for match in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[match.trainIdx].pt for match in matches]).reshape(-1, 1, 2)

    # Find the homography matrix
    H, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)

    # Warp the current image to the panorama
    warped_image = cv.warpPerspective(images[i], H, (stitched_image.shape[1] + images[i].shape[1], images[i].shape[0]))

    # Blend the images using simple averaging or other methods
    stitched_image = cv.addWeighted(stitched_image, 0.5, warped_image, 0.5, 0)

# Display or save the stitched panorama
cv.imshow("Stitched Panorama", stitched_image)
cv.waitKey(0)
cv.destroyAllWindows()
